# Search engine demonstration

## Βήμα 1
Αρχικά ξεκινάμε τρέχοντας μέσω CMD τις επόμενες εντολές:
- pip install notebook
- pip install beautifulsoup4
- pip install requests
- pip install pandas

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import re

# Λίστα με τα 10 URLs
urls = [
    'https://en.wikipedia.org/wiki/Python_(programming_language)',
    'https://en.wikipedia.org/wiki/JavaScript',
    'https://en.wikipedia.org/wiki/Java_(programming_language)',
    'https://en.wikipedia.org/wiki/C_(programming_language)',
    'https://en.wikipedia.org/wiki/Ruby_(programming_language)',
    'https://en.wikipedia.org/wiki/HTML',
    'https://en.wikipedia.org/wiki/C%2B%2B',
    'https://en.wikipedia.org/wiki/Go_(programming_language)',
    'https://en.wikipedia.org/wiki/Swift_(programming_language)',
    'https://en.wikipedia.org/wiki/Kotlin'
]

# Συνάρτηση για να τραβήξουμε δεδομένα από κάθε URL
def crawl_page(url):
    print(f"Crawling: {url}")
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Εξαγωγή όλων των λέξεων από την σελίδα
            # Επιλέγουμε τα tags που περιέχουν κείμενο: p (paragraphs), li (list items), span, a (links) και άλλα
            text = []
            for tag in soup.find_all(['p', 'li', 'span', 'a', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6']):
                text.append(tag.get_text())
            
            # Συνενώνουμε όλα τα κείμενα και καθαρίζουμε από περιττά κενά
            full_text = ' '.join(text)
            # Καθαρισμός κειμένου (π.χ. αφαιρούμε περιττές λευκές περιοχές)
            cleaned_text = re.sub(r'\s+', ' ', full_text).strip()
            
            return cleaned_text
        else:
            print(f"Failed to retrieve {url}")
            return ""
    except Exception as e:
        print(f"Error while crawling {url}: {e}")
        return ""

# Δημιουργία λεξικού για αποθήκευση του πλήρους κειμένου ανά URL
text_dict = {}

# Κάνουμε crawl τις 10 σελίδες
for url in urls:
    page_text = crawl_page(url)
    text_dict[url] = page_text

# Αποθήκευση των δεδομένων σε αρχείο JSON
with open('programming_languages_text.json', 'w') as file:
    json.dump(text_dict, file, indent=4)

print("Data saved to programming_languages_text.json")


Ο παραπάνω κώδικας υλοποιεί μια διαδικασία web scraping για να τραβήξει κείμενο από 10 σελίδες της Wikipedia που σχετίζονται με γλώσσες προγραμματισμού. Αναλυτικά:

1. **Λίστα URLs**: Ορίζεται μια λίστα με 10 URLs που περιέχουν πληροφορίες για γλώσσες προγραμματισμού, όπως Python, Java, C, HTML κ.λπ.

2. **Συνάρτηση crawl_page**:
- Κάνει HTTP αίτημα (GET) σε κάθε URL.
- Αν η απάντηση είναι επιτυχής (κωδικός 200), χρησιμοποιεί το BeautifulSoup για να αναλύσει το HTML περιεχόμενο της σελίδας.
- Εξάγει κείμενο από διάφορα HTML tags όπως *p, li, span, a, h1-h6*.
- Συνενώνει το κείμενο από όλα τα tags σε μια ενιαία συμβολοσειρά.
- Καθαρίζει το κείμενο από περιττά κενά και επιστρέφει το τελικό καθαρό κείμενο.

3. **Αποθήκευση Δεδομένων**:
- Τα δεδομένα αποθηκεύονται σε ένα λεξικό text_dict όπου το κλειδί είναι το URL και η τιμή είναι το εξαγόμενο κείμενο από τη σελίδα.
- Τέλος, τα δεδομένα αποθηκεύονται σε αρχείο JSON με το όνομα *programming_languages_text.json*.

4. **Αρχειοθέτηση**:
Το JSON αρχείο που δημιουργείται, περιέχει τα κείμενα από όλες τις σελίδες, τα οποία μπορούν να χρησιμοποιηθούν για περαιτέρω επεξεργασία, όπως δημιουργία ενός inverted index ή κατάταξη εγγράφων.

![step 1](./project/images/step_1.png) 

## Βήμα 2

- **Κατέβασμα των απαιτούμενων πόρων του NLTK**: 
Χρησιμοποιούμε το NLTK για να κατεβάσουμε τα δεδομένα που απαιτούνται για την επεξεργασία του κειμένου, όπως οι stopwords και η λέξη-ρίζα λεξικογράφηση.
- **Φόρτωση stopwords**: 
Οι stopwords είναι κοινές λέξεις που συνήθως αφαιρούνται κατά την επεξεργασία φυσικής γλώσσας, όπως "the", "is", κλπ.
- **Αρχικοποίηση του Stemming και Lemmatization**: 
ο stemming απομακρύνει τα κατάληξη των λέξεων για να επιστρέψει τη ρίζα της λέξης, ενώ το lemmatization επιστρέφει την κανονική μορφή της λέξης.
- **Διαδικασία καθαρισμού**: 
Ο καθαρισμός του κειμένου περιλαμβάνει το tokenization, την αφαίρεση stopwords, την αφαίρεση της στίξης, και την εφαρμογή του stemming και του lemmatization.
- **Αποθήκευση των επεξεργασμένων δεδομένων**: 
Τα επεξεργασμένα δεδομένα αποθηκεύονται σε ένα νέο αρχείο JSON.

In [ ]:
import json
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Κατεβάζουμε τους απαιτούμενους πόρους του NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Ορίζουμε τις διαδρομές των αρχείων εισόδου και εξόδου
input_json_path = 'programming_languages_text.json'
output_json_path = 'processed_programming_languages_text.json'

# Φορτώνουμε τις λέξεις κλειδιά (stopwords)
stop_words = set(stopwords.words('english'))

# Αρχικοποιούμε τον αλγόριθμο stemming και lemmatization
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

# Debug: Εμφανίζουμε τις stopwords που χρησιμοποιούνται
print("Stopwords used:", stop_words)

# Ορίζουμε μία συνάρτηση για τον καθαρισμό του κειμένου
def clean_text(text):
    # Κάνουμε tokenization του κειμένου σε λέξεις
    words = word_tokenize(text)
    
    # Αφαιρούμε τις stopwords και την στίξη
    cleaned_words = [
        word.lower() for word in words 
        if word.lower() not in stop_words and word not in string.punctuation
    ]
    
    # Εφαρμόζουμε το stemming
    stemmed_words = [stemmer.stem(word) for word in cleaned_words]
    
    # Εφαρμόζουμε το lemmatization
    lemmatized_words = [lemmatizer.lemmatize(word) for word in stemmed_words]
    
    # Ενώνουμε τις καθαρές, stemmed και lemmatized λέξεις πίσω σε μία συμβολοσειρά
    return ' '.join(lemmatized_words)

# Διαβάζουμε το αρχείο εισόδου JSON
with open(input_json_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# Επεξεργαζόμαστε τα δεδομένα κειμένου και αποθηκεύουμε την καθαρισμένη εκδοχή μόνο
processed_data = {}
for key, value in data.items():
    processed_value = clean_text(value)
    processed_data[key] = processed_value

# Αποθηκεύουμε τα επεξεργασμένα δεδομένα στο αρχείο εξόδου JSON
with open(output_json_path, 'w', encoding='utf-8') as file:
    json.dump(processed_data, file, ensure_ascii=False, indent=4)

print(f"Τα επεξεργασμένα δεδομένα αποθηκεύτηκαν στο {output_json_path}")


![step2](./project/images/step_2.png)

## Βήμα 3

Για το βήμα 3, χρειάστηκε να εκτελέσουμε την παρακάτω εντολή (για τη βιβλιοθήκη nltk):
- pip install nltk
##
- **Διαβάζουμε τα προεπεξεργασμένα δεδομένα**: Ανοίγουμε το αρχείο JSON που περιέχει τα επεξεργασμένα δεδομένα κειμένου που έχουμε αποθηκεύσει προηγουμένως.

- **Αρχικοποιούμε το ανεστραμμένο ευρετήριο (Inverted Index)**: Το ανεστραμμένο ευρετήριο είναι μία δομή δεδομένων που μας επιτρέπει να εντοπίσουμε γρήγορα σε ποια έγγραφα εμφανίζεται κάθε λέξη.

- **Δημιουργία του ανεστραμμένου ευρετηρίου**: Για κάθε κείμενο που έχουμε επεξεργαστεί, διαχωρίζουμε το κείμενο σε λέξεις και προσθέτουμε την κάθε λέξη στο ανεστραμμένο ευρετήριο. Κάθε λέξη θα δείχνει σε ποια έγγραφα (ταυτοποιημένα με doc_id) εμφανίζεται.

- **Αποθήκευση του ανεστραμμένου ευρετηρίου**: Αποθηκεύουμε το ανεστραμμένο ευρετήριο σε ένα νέο αρχείο JSON για μελλοντική χρήση.


In [ ]:
import json

# Διαβάζουμε τα προεπεξεργασμένα δεδομένα
with open('processed_programming_languages_text.json', 'r', encoding='utf-8') as file:
    processed_data = json.load(file)

# Αρχικοποιούμε το ανεστραμμένο ευρετήριο (Inverted Index)
inverted_index = {}

# Δημιουργούμε το ανεστραμμένο ευρετήριο
for doc_id, text in processed_data.items():
    # Διαχωρίζουμε το κείμενο σε λέξεις
    words = text.split()
    
    for word in words:
        # Προσθέτουμε τη λέξη στο ανεστραμμένο ευρετήριο
        if word not in inverted_index:
            inverted_index[word] = []  # Αρχικοποιούμε μία κενή λίστα για τη λέξη
        if doc_id not in inverted_index[word]:
            inverted_index[word].append(doc_id)

# Αποθηκεύουμε το ανεστραμμένο ευρετήριο σε αρχείο
output_index_path = 'inverted_index.json'
with open(output_index_path, 'w', encoding='utf-8') as file:
    json.dump(inverted_index, file, ensure_ascii=False, indent=4)

print(f"Το ανεστραμμένο ευρετήριο αποθηκεύτηκε στο {output_index_path}")



![step 3](./project/images/step_3.png)

## Βήμα 4

- **Διαβάζουμε το ανεστραμμένο ευρετήριο**: Φορτώνουμε το ανεστραμμένο ευρετήριο από το αρχείο JSON που δημιουργήσαμε προηγουμένως.

- **Επεξεργασία Boolean Query**: Η συνάρτηση boolean_query_processor επεξεργάζεται ένα Boolean query (όπως "python AND java") και επιστρέφει τα έγγραφα που ταιριάζουν με την αναζήτηση. Χρησιμοποιεί τελεστές όπως "AND", "OR", "NOT".

- **Διαχωρισμός του query σε όρους και τελεστές**: Στην συνάρτηση parse_query, το query διαχωρίζεται σε όρους και τελεστές (AND, OR, NOT). Αυτοί οι τελεστές χρησιμοποιούνται για να συνδυαστούν τα αποτελέσματα από το ανεστραμμένο ευρετήριο.

- **Αναζήτηση για κάθε όρο**: Η συνάρτηση get_documents επιστρέφει τα έγγραφα που περιέχουν κάθε όρο από το ανεστραμμένο ευρετήριο.

- **Συνδυασμός των αποτελεσμάτων με βάση τους τελεστές**: Χρησιμοποιούμε τελεστές για να συνδυάσουμε τα αποτελέσματα των όρων και να υπολογίσουμε ποια έγγραφα ταιριάζουν με το query. Αν ο τελεστής είναι "AND", παίρνουμε τα κοινά έγγραφα (διασταύρωση), αν είναι "OR", παίρνουμε τη ένωση, και αν είναι "NOT", παίρνουμε τη διαφορά.

- **Παράδειγμα ελέγχου**: Ενδεικτικά, εκτελούμε το query "python AND java" και εκτυπώνουμε τα έγγραφα που ταιριάζουν με το query.





*boolean_query_processor.py*

In [ ]:
import json

# Διαβάζουμε το ανεστραμμένο ευρετήριο
with open('inverted_index.json', 'r', encoding='utf-8') as file:
    inverted_index = json.load(file)

def boolean_query_processor(query, inverted_index):
    """
    Επεξεργάζεται ένα Boolean query και επιστρέφει τα αναγνωριστικά των εγγράφων που ταιριάζουν.
    """
    def parse_query(query):
        # Διαχωρίζουμε το query σε όρους και τελεστές
        tokens = query.split()
        terms = []
        operators = []
        for token in tokens:
            if token in {"AND", "OR", "NOT"}:
                operators.append(token)
            else:
                terms.append(token)
        return terms, operators

    def get_documents(term):
        # Παίρνουμε τα έγγραφα για έναν όρο από το ανεστραμμένο ευρετήριο
        return set(inverted_index.get(term, []))

    terms, operators = parse_query(query)
    if not terms:
        return set()  # Αν δεν υπάρχουν όροι, επιστρέφουμε κενό σύνολο

    # Αρχικοποιούμε το σύνολο των αποτελεσμάτων με τον πρώτο όρο
    result = get_documents(terms[0])

    # Επεξεργαζόμαστε τους τελεστές και συνδυάζουμε τα αποτελέσματα
    for i, operator in enumerate(operators):
        next_term_docs = get_documents(terms[i + 1])
        if operator == "AND":
            result = result.intersection(next_term_docs)  # Διασταύρωση (AND)
        elif operator == "OR":
            result = result.union(next_term_docs)  # Ένωση (OR)
        elif operator == "NOT":
            result = result.difference(next_term_docs)  # Διαφορά (NOT)

    return result

# Παράδειγμα ελέγχου
query = "python AND java"
matching_docs = boolean_query_processor(query, inverted_index)
print(f"Έγγραφα που ταιριάζουν με το '{query}': {matching_docs}")



![step 4i](./project/images/step_4_i.png)

---
- **Διαβάζουμε το ανεστραμμένο ευρετήριο**: Ανοίγουμε το αρχείο *inverted_index.json* και φορτώνουμε το αντίστροφο ευρετήριο.

- **Διαβάζουμε τα επεξεργασμένα δεδομένα**: Φορτώνουμε το αρχείο με τα προεπεξεργασμένα δεδομένα κειμένου (*processed_programming_languages_text.json*).

- **Συνάρτηση για να βρούμε τα έγγραφα που ταιριάζουν με το query**: Η συνάρτηση `get_matching_docs` παίρνει το query, το διαχωρίζει σε όρους και βρίσκει ποια έγγραφα περιέχουν αυτούς τους όρους με τη βοήθεια του ανεστραμμένου ευρετηρίου.

- **Συνάρτηση για να υπολογίσουμε το TF-IDF και να κατατάξουμε τα έγγραφα**: Η συνάρτηση `compute_tf_idf` υπολογίζει την ομοιότητα συνημίτονου (cosine similarity) ανάμεσα στο query και τα έγγραφα. Αυτό γίνεται μέσω του υπολογισμού του TF-IDF των λέξεων του query και των εγγράφων.

- **Εμφάνιση των αποτελεσμάτων**: Τα έγγραφα κατατάσσονται με βάση την ομοιότητα τους με το query και εμφανίζονται στον χρήστη.

*tfidf_ranking.py*

In [ ]:
import json
from sklearn.feature_extraction.text import TfidfVectorizer

# Διαβάζουμε το ανεστραμμένο ευρετήριο
with open('inverted_index.json', 'r', encoding='utf-8') as file:
    inverted_index = json.load(file)

# Διαβάζουμε τα επεξεργασμένα δεδομένα
with open('processed_programming_languages_text.json', 'r', encoding='utf-8') as file:
    processed_data = json.load(file)

# Συνάρτηση για να βρούμε τα έγγραφα που ταιριάζουν με το query
def get_matching_docs(query):
    query_terms = query.lower().split()  # Διαχωρίζουμε το query σε όρους
    matching_docs = set()

    # Ελέγχουμε ποια έγγραφα ταιριάζουν με τους όρους του query
    for term in query_terms:
        if term in inverted_index:
            matching_docs.update(inverted_index[term])

    return list(matching_docs)

# Συνάρτηση για να υπολογίσουμε το TF-IDF και να κατατάξουμε τα έγγραφα
def compute_tf_idf(query, matching_docs, processed_data):
    # Προετοιμάζουμε το corpus: περιλαμβάνουμε μόνο τα έγγραφα που ταιριάζουν
    corpus = [processed_data[doc_id] for doc_id in matching_docs]
    
    # Προσθέτουμε το query ως τελευταίο στοιχείο στο corpus
    corpus.append(query)

    # Χρησιμοποιούμε τον TfidfVectorizer
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(corpus)

    # Παίρνουμε το vector του query (τελευταία γραμμή του matrix)
    query_vector = tfidf_matrix[-1]

    # Υπολογίζουμε την ομοιότητα συνημίτονου (cosine similarity) μεταξύ του query και κάθε εγγράφου
    cosine_similarities = (tfidf_matrix[:-1] @ query_vector.T).toarray().flatten()

    # Ταιριάζουμε κάθε έγγραφο με την βαθμολογία της ομοιότητας
    ranked_docs = sorted(
        zip(matching_docs, cosine_similarities),
        key=lambda x: x[1],
        reverse=True
    )

    return ranked_docs

if __name__ == "__main__":
    # Ζητάμε από τον χρήστη να εισάγει ένα query
    query = input("Εισάγετε το query σας (κατάταξη TF-IDF): ").strip()

    # Βρίσκουμε τα έγγραφα που ταιριάζουν με το query
    matching_docs = get_matching_docs(query)
    
    if not matching_docs:
        print("Δεν βρέθηκαν έγγραφα που να ταιριάζουν.")
    else:
        # Υπολογίζουμε και κατατάσσουμε τα έγγραφα με βάση το TF-IDF
        ranked_docs = compute_tf_idf(query, matching_docs, processed_data)

        # Εμφανίζουμε τα αποτελέσματα
        print("\nΚαταταγμένα αποτελέσματα:")
        for doc_id, score in ranked_docs:
            print(f"{doc_id}: {score:.4f}")


![step 4ii](./project/images/step_4_ii.png)


---
- **Διαβάζουμε το ανεστραμμένο ευρετήριο**: Φορτώνουμε το ανεστραμμένο ευρετήριο από το αρχείο *inverted_index.json*.

- **Διαβάζουμε τα επεξεργασμένα δεδομένα**: Φορτώνουμε το αρχείο με τα προεπεξεργασμένα δεδομένα κειμένου από το αρχείο *processed_programming_languages_text.json*.

- **Συνάρτηση για κατάταξη με BM25**: Η συνάρτηση `bm25_ranking` χρησιμοποιεί τον αλγόριθμο BM25 για να κατατάξει τα έγγραφα με βάση την ομοιότητα του query με τα έγγραφα. Για κάθε έγγραφο, υπολογίζουμε τη βαθμολογία του και τα ταξινομούμε σε φθίνουσα σειρά.

- **Κύρια συνάρτηση της μηχανής αναζήτησης**: Η συνάρτηση `search_engine` είναι η κεντρική λειτουργία της μηχανής αναζήτησης. Επιτρέπει στον χρήστη να εισάγει ένα query και να επιλέξει τη μέθοδο κατάταξης (Boolean, TF-IDF ή BM25). Παρουσιάζει τα αποτελέσματα του query με την επιλεγμένη μέθοδο κατάταξης.

- **Επιλογή μεθόδου κατάταξης**: Ο χρήστης μπορεί να επιλέξει την μέθοδο κατάταξης (Boolean, TF-IDF, BM25) για τα αποτελέσματα της αναζήτησης.

- **Εμφάνιση καταταγμένων αποτελεσμάτων**: Τα αποτελέσματα του query εμφανίζονται με την επιλεγμένη μέθοδο κατάταξης και κάθε έγγραφο εμφανίζεται με τη σχετική του βαθμολογία.

*search_engine.py*

In [ ]:
import json
from tfidf_ranking import compute_tf_idf
from boolean_query_processor import boolean_query_processor
from rank_bm25 import BM25Okapi

# Διαβάζουμε το ανεστραμμένο ευρετήριο
with open("inverted_index.json", "r", encoding="utf-8") as f:
    inverted_index = json.load(f)

# Διαβάζουμε τα επεξεργασμένα δεδομένα
with open("processed_programming_languages_text.json", "r", encoding="utf-8") as f:
    processed_data = json.load(f)

# Συνάρτηση για κατάταξη με BM25
def bm25_ranking(query, matching_docs, processed_data):
    # Tokenize (διαχωρίζουμε σε λέξεις) τα έγγραφα
    tokenized_docs = [doc.split() for doc in processed_data.values()]
    
    # Δημιουργούμε το αντικείμενο BM25
    bm25 = BM25Okapi(tokenized_docs)
    
    # Διαχωρίζουμε το query σε λέξεις
    query_tokens = query.split()
    
    # Υπολογίζουμε τις βαθμολογίες (scores) του query
    scores = bm25.get_scores(query_tokens)
    
    # Συνδυάζουμε τα έγγραφα με τις αντίστοιχες βαθμολογίες τους
    doc_score_pairs = [(doc, score) for doc, score in zip(processed_data.keys(), scores) if doc in matching_docs]
    
    # Ταξινομούμε τα έγγραφα κατά φθίνουσα βαθμολογία
    ranked_results = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)
    
    return [doc for doc, _ in ranked_results]

# Κύρια συνάρτηση της μηχανής αναζήτησης
def search_engine():
    print("Καλώς ήρθατε στην Μηχανή Αναζήτησης!")
    print("Πληκτρολογήστε το query σας χρησιμοποιώντας Boolean τελεστές (AND, OR, NOT).")
    print("Παράδειγμα: python AND java OR javascript\n")
    
    while True:
        query = input("Εισάγετε το query σας (ή πληκτρολογήστε 'exit' για έξοδο): ").strip()
        if query.lower() == "exit":
            print("Αντίο!")
            break

        # Επεξεργαζόμαστε το query χρησιμοποιώντας την Boolean αναζήτηση
        matching_docs = boolean_query_processor(query, inverted_index)
        if not matching_docs:
            print(f"Δεν βρέθηκαν έγγραφα που να ταιριάζουν με το query: {query}")
            continue

        print(f"\nΈγγραφα που ταιριάζουν με το '{query}': {matching_docs}")

        # Επιλογή μεθόδου κατάταξης από τον χρήστη
        print("\nΕπιλέξτε μέθοδο κατάταξης:")
        print("1. Boolean Αναζήτηση (Προεπιλογή)")
        print("2. Κατάταξη TF-IDF")
        print("3. Κατάταξη BM25")
        choice = input("Εισάγετε τον αριθμό της επιλογής σας (1, 2 ή 3): ").strip()

        if choice == "2":
            ranked_docs = compute_tf_idf(query, matching_docs, processed_data)
            print("\nΚαταταγμένα αποτελέσματα με TF-IDF:")
        elif choice == "3":
            ranked_docs = bm25_ranking(query, matching_docs, processed_data)
            print("\nΚαταταγμένα αποτελέσματα με BM25:")
        else:
            ranked_docs = list(matching_docs)
            print("\nΑποτελέσματα Boolean Αναζήτησης:")

        # Εμφάνιση των καταταγμένων αποτελεσμάτων
        for rank, doc in enumerate(ranked_docs, start=1):
            print(f"{rank}. {doc}")

# Εκκίνηση της μηχανής αναζήτησης
if __name__ == "__main__":
    search_engine()

Ακολουθεί ενδεικτική εκτέλεση.








C:\Users\Gregory\Desktop\project>python search_engine.py
Documents matching 'python AND java': {'https://en.wikipedia.org/wiki/Ruby_(programming_language)', 'https://en.wikipedia.org/wiki/Java_(programming_language)', 'https://en.wikipedia.org/wiki/C_(programming_language)', 'https://en.wikipedia.org/wiki/Python_(programming_language)', 'https://en.wikipedia.org/wiki/Swift_(programming_language)', 'https://en.wikipedia.org/wiki/JavaScript', 'https://en.wikipedia.org/wiki/C%2B%2B', 'https://en.wikipedia.org/wiki/Go_(programming_language)'}
Welcome to the Search Engine!
Type your query using Boolean operators (AND, OR, NOT).
Example: python AND java OR javascript

Enter your query (or type 'exit' to quit): java or hello

Documents matching 'java or hello': {'https://en.wikipedia.org/wiki/Ruby_(programming_language)', 'https://en.wikipedia.org/wiki/Java_(programming_language)', 'https://en.wikipedia.org/wiki/C_(programming_language)', 'https://en.wikipedia.org/wiki/Python_(programming_language)', 'https://en.wikipedia.org/wiki/Swift_(programming_language)', 'https://en.wikipedia.org/wiki/JavaScript', 'https://en.wikipedia.org/wiki/C%2B%2B', 'https://en.wikipedia.org/wiki/Go_(programming_language)'}

Choose a ranking method:
1. Boolean Retrieval (Default)
2. TF-IDF Ranking
3. BM25 Ranking
Enter the number of your choice (1, 2, or 3): 1

Boolean Retrieval Results:
1. https://en.wikipedia.org/wiki/Ruby_(programming_language)
2. https://en.wikipedia.org/wiki/Java_(programming_language)
3. https://en.wikipedia.org/wiki/C_(programming_language)
4. https://en.wikipedia.org/wiki/Python_(programming_language)
5. https://en.wikipedia.org/wiki/Swift_(programming_language)
6. https://en.wikipedia.org/wiki/JavaScript
7. https://en.wikipedia.org/wiki/C%2B%2B
8. https://en.wikipedia.org/wiki/Go_(programming_language)
Enter your query (or type 'exit' to quit): java or hello

Documents matching 'java or hello': {'https://en.wikipedia.org/wiki/Ruby_(programming_language)', 'https://en.wikipedia.org/wiki/Java_(programming_language)', 'https://en.wikipedia.org/wiki/C_(programming_language)', 'https://en.wikipedia.org/wiki/Python_(programming_language)', 'https://en.wikipedia.org/wiki/Swift_(programming_language)', 'https://en.wikipedia.org/wiki/JavaScript', 'https://en.wikipedia.org/wiki/C%2B%2B', 'https://en.wikipedia.org/wiki/Go_(programming_language)'}

Choose a ranking method:
1. Boolean Retrieval (Default)
2. TF-IDF Ranking
3. BM25 Ranking
Enter the number of your choice (1, 2, or 3): 2

TF-IDF Ranked Results:
1. ('https://en.wikipedia.org/wiki/Java_(programming_language)', np.float64(0.25966958427207704))
2. ('https://en.wikipedia.org/wiki/JavaScript', np.float64(0.01689162517913656))
3. ('https://en.wikipedia.org/wiki/C_(programming_language)', np.float64(0.015636927175054036))
4. ('https://en.wikipedia.org/wiki/C%2B%2B', np.float64(0.01499599173627299))
5. ('https://en.wikipedia.org/wiki/Go_(programming_language)', np.float64(0.012284060668750124))
6. ('https://en.wikipedia.org/wiki/Ruby_(programming_language)', np.float64(0.007483638885609637))
7. ('https://en.wikipedia.org/wiki/Swift_(programming_language)', np.float64(0.006847389333614802))
8. ('https://en.wikipedia.org/wiki/Python_(programming_language)', np.float64(0.005532624249020468))
Enter your query (or type 'exit' to quit): java or hello

Documents matching 'java or hello': {'https://en.wikipedia.org/wiki/Ruby_(programming_language)', 'https://en.wikipedia.org/wiki/Java_(programming_language)', 'https://en.wikipedia.org/wiki/C_(programming_language)', 'https://en.wikipedia.org/wiki/Python_(programming_language)', 'https://en.wikipedia.org/wiki/Swift_(programming_language)', 'https://en.wikipedia.org/wiki/JavaScript', 'https://en.wikipedia.org/wiki/C%2B%2B', 'https://en.wikipedia.org/wiki/Go_(programming_language)'}

Choose a ranking method:
1. Boolean Retrieval (Default)
2. TF-IDF Ranking
3. BM25 Ranking
Enter the number of your choice (1, 2, or 3): 3

BM25 Ranked Results:
1. https://en.wikipedia.org/wiki/Java_(programming_language)
2. https://en.wikipedia.org/wiki/C_(programming_language)
3. https://en.wikipedia.org/wiki/C%2B%2B
4. https://en.wikipedia.org/wiki/JavaScript
5. https://en.wikipedia.org/wiki/Go_(programming_language)
6. https://en.wikipedia.org/wiki/Swift_(programming_language)
7. https://en.wikipedia.org/wiki/Python_(programming_language)
8. https://en.wikipedia.org/wiki/Ruby_(programming_language)

## Βήμα 5

SearchEngineEvaluator

1. **Φόρτωση του Αντίστροφου Ευρετηρίου**:

Η πρώτη συνάρτηση, *load_inverted_index*, φορτώνει το αντίστροφο ευρετήριο από ένα αρχείο JSON. Αν το αρχείο δεν υπάρχει, η συνάρτηση επιστρέφει None και εκτυπώνει μήνυμα λάθους. Διαφορετικά, το αρχείο διαβάζεται και επιστρέφεται το αντίστροφο ευρετήριο.

In [ ]:
import json
import os

# Φόρτωση του ανεστραμμένου ευρετηρίου από το αρχείο JSON
def load_inverted_index(file_path):
    if not os.path.exists(file_path):
        print(f"Error: File {file_path} not found.")
        return None
    with open(file_path, 'r', encoding='utf-8') as f:
        inverted_index = json.load(f)
    return inverted_index

2. **Επεξεργασία Boolean Queries**:

Η συνάρτηση `boolean_query_processor` είναι υπεύθυνη για την επεξεργασία του Boolean query που παρέχεται από τον χρήστη. Αν το query περιέχει τις λέξεις κλειδιά AND, OR, ή NOT, η συνάρτηση θα επεξεργαστεί τα αντίστοιχα αποτελέσματα χρησιμοποιώντας τις αντίστοιχες λογικές πράξεις. Αν το query δεν περιέχει κανένα από αυτά τα λογικά τελεστικά, απλώς επιστρέφει τα έγγραφα που περιέχουν τον πρώτο όρο του query.

- Αν το query περιέχει AND, επιστρέφονται τα έγγραφα που περιέχουν όλους τους όρους του query.
- Αν το query περιέχει OR, επιστρέφονται τα έγγραφα που περιέχουν τουλάχιστον έναν από τους όρους.
- Αν το query περιέχει NOT, επιστρέφονται τα έγγραφα που περιέχουν τον πρώτο όρο, αλλά χωρίς τους υπόλοιπους.

In [ ]:
# Συνάρτηση για την επεξεργασία Boolean queries και την επιστροφή των ταιριαστών εγγράφων
def boolean_query_processor(query, inverted_index):
    query_terms = query.lower().split(' ')
    if 'and' in query_terms:
        terms = [term for term in query_terms if term != 'and']
        matching_docs = set(inverted_index.get(terms[0], []))
        for term in terms[1:]:
            matching_docs &= set(inverted_index.get(term, []))
    elif 'or' in query_terms:
        terms = [term for term in query_terms if term != 'or']
        matching_docs = set(inverted_index.get(terms[0], []))
        for term in terms[1:]:
            matching_docs |= set(inverted_index.get(term, []))
    elif 'not' in query_terms:
        terms = [term for term in query_terms if term != 'not']
        matching_docs = set(inverted_index.get(terms[0], []))
        for term in terms[1:]:
            matching_docs -= set(inverted_index.get(term, []))
    else:
        matching_docs = set(inverted_index.get(query_terms[0], []))

    return matching_docs


3. **Υπολογισμός Ακρίβειας, Ανάκτησης και F1-Score**:

Η συνάρτηση `evaluate_metrics` υπολογίζει την ακρίβεια, την ανάκτηση και το F1-Score για τα έγγραφα που ταιριάζουν με το query. Αυτά τα μέτρα χρησιμοποιούνται συχνά για την αξιολόγηση της απόδοσης ενός συστήματος αναζήτησης:

- **Ακρίβεια (Precision)**: Το ποσοστό των ταιριαστών εγγράφων που είναι πραγματικά συναφή με το query.
- **Ανάκτηση (Recall)**: Το ποσοστό των συναφών εγγράφων που ανακτήθηκαν από το σύστημα.
- **F1-Score**: Το αρμονικό μέσο της ακρίβειας και της ανάκτησης, το οποίο προσφέρει μια ενιαία μέτρηση.


In [ ]:
# Συνάρτηση για τον υπολογισμό της ακρίβειας, ανάκτησης και F1-Score
def evaluate_metrics(matching_docs, relevant_docs):
    relevant_doc_keys = set(relevant_docs.keys())
    
    tp = len(matching_docs & relevant_doc_keys)
    fp = len(matching_docs - relevant_doc_keys)
    fn = len(relevant_doc_keys - matching_docs)
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
    
    return precision, recall, f1_score

4. **Υπολογισμός του Mean Average Precision (MAP)**:

Η συνάρτηση `mean_average_precision` υπολογίζει το Mean Average Precision για μια σειρά από queries. Ο μέσος όρος των AP (Average Precision) για κάθε query υπολογίζεται και χρησιμοποιείται για να πάρουμε το MAP score.

- **Average Precision (AP)**: Ο υπολογισμός της ακρίβειας για κάθε θέση στο σύνολο των αποτελεσμάτων, με βάση τη σειρά κατάταξης.

In [ ]:
# Συνάρτηση για τον υπολογισμό του Mean Average Precision (MAP)
def mean_average_precision(query_results, relevant_docs_for_queries):
    average_precision_scores = []
    for query, retrieved_docs in query_results.items():
        relevant_docs = relevant_docs_for_queries.get(query, {})
        ap = average_precision(retrieved_docs, relevant_docs)
        average_precision_scores.append(ap)
    
    map_score = sum(average_precision_scores) / len(average_precision_scores) if average_precision_scores else 0.0
    return map_score

5. **Υπολογισμός του Average Precision (AP)**: 

Η συνάρτηση `average_precision` υπολογίζει το AP για κάθε query, μετρώντας την ακρίβεια σε κάθε θέση στην κατάταξη των αποτελεσμάτων και παίρνοντας τον μέσο όρο αυτών των τιμών.

In [ ]:
# Συνάρτηση για τον υπολογισμό του Average Precision (AP) για ένα μόνο query
def average_precision(retrieved_docs, relevant_docs):
    retrieved_docs = list(retrieved_docs)
    relevant_docs_set = set(relevant_docs.keys())
    relevant_count = 0
    precision_at_k = []
    
    for i, doc in enumerate(retrieved_docs, start=1):
        if doc in relevant_docs_set:
            relevant_count += 1
            precision_at_k.append(relevant_count / i)
    
    ap = sum(precision_at_k) / len(relevant_docs) if relevant_docs else 0.0
    return ap

6. **Αξιολόγηση της Μηχανής Αναζήτησης**:

Η συνάρτηση `evaluate_search_engine` αξιολογεί τη μηχανή αναζήτησης για μια σειρά από queries. Για κάθε query, χρησιμοποιείται η συνάρτηση `boolean_query_processor` για να βρούμε τα έγγραφα που ταιριάζουν με το query και στη συνέχεια υπολογίζονται οι επιδόσεις (precision, recall, F1-Score).

Επίσης, αποθηκεύονται τα αποτελέσματα των queries για τον υπολογισμό του Mean Average Precision (MAP).

In [ ]:
# Συνάρτηση για την αξιολόγηση της μηχανής αναζήτησης για μια σειρά queries
def evaluate_search_engine(inverted_index):
    queries = [
        "python AND java",
        "content OR jump",
        "python NOT java"
    ]
    
    relevant_docs_for_queries = {
        "python AND java": {
            "https://en.wikipedia.org/wiki/Python_(programming_language)": 1,
            "https://en.wikipedia.org/wiki/Java_(programming_language)": 1
        },
        "content OR jump": {
            "https://en.wikipedia.org/wiki/Python_(programming_language)": 1,
            "https://en.wikipedia.org/wiki/JavaScript": 1
        },
        "python NOT java": {
            "https://en.wikipedia.org/wiki/Python_(programming_language)": 1
        }
    }

    query_results = {}
    print("Αποτελέσματα Αξιολόγησης:\n")
    for query in queries:
        print(f"Ερώτημα: {query}")
        matching_docs = boolean_query_processor(query, inverted_index)
        relevant_docs = relevant_docs_for_queries.get(query, {})
        precision, recall, f1_score = evaluate_metrics(matching_docs, relevant_docs)
        
        print(f"Ταιριαστά Έγγραφα: {matching_docs}")
        print(f"Ακρίβεια: {precision:.2f}")
        print(f"Ανάκτηση: {recall:.2f}")
        print(f"F1-Score: {f1_score:.2f}\n")
        
        # Αποθήκευση των αποτελεσμάτων των queries για τον υπολογισμό του MAP
        query_results[query] = matching_docs

    # Υπολογισμός του Mean Average Precision (MAP)
    map_score = mean_average_precision(query_results, relevant_docs_for_queries)
    print(f"Mean Average Precision (MAP): {map_score:.2f}")

7. **Κύρια Συνάρτηση**:

Η κύρια συνάρτηση `main` φορτώνει το ανεστραμμένο ευρετήριο από το αρχείο *inverted_index.json* και αν είναι έγκυρο, καλεί τη συνάρτηση `evaluate_search_engine` για να αξιολογήσει τη μηχανή αναζήτησης.

In [ ]:
# Κύρια συνάρτηση για τη φόρτωση του ανεστραμμένου ευρετηρίου και την εκτέλεση της αξιολόγησης
def main():
    inverted_index = load_inverted_index('inverted_index.json')
    if inverted_index is not None:
        evaluate_search_engine(inverted_index)

if __name__ == "__main__":
    main()

![step 5](./project/images/step_5.png)